In [13]:
import pandas as pd
import networkx as nx
import plotly.graph_objects as go
import numpy as np

# Cargar dataset
url = '/content/proyectos_criptos_categorias.csv'
df = pd.read_csv(url)

# Crear un grafo
B = nx.Graph()

# Conjuntos de nodos: criptomonedas y categorías
cryptos = df['symbol'].unique()
categories = df['category'].unique()

# Añadir nodos al grafo
B.add_nodes_from(cryptos, bipartite=0)  # Nodos criptomonedas
B.add_nodes_from(categories, bipartite=1)  # Nodos categorías

# Añadir aristas con ponderaciones entre criptomonedas y categorías
for index, row in df.iterrows():
    B.add_edge(row['symbol'], row['category'], weight=row['volume_24h'])  # Ponderación según volumen

# Posicionamiento de los nodos
# Colocar las criptomonedas en un círculo alrededor
pos = nx.circular_layout(cryptos)

# Colocar las categorías distribuidas en un círculo más pequeño en el centro
angle_step = 2 * np.pi / len(categories)  # Separar las categorías en el centro
radius = 0.2  # Radio más pequeño para las categorías en el centro
for i, category in enumerate(categories):
    pos[category] = [radius * np.cos(i * angle_step), radius * np.sin(i * angle_step)]

# Extraer posiciones para plotly
x_nodes = [pos[node][0] for node in B.nodes()]
y_nodes = [pos[node][1] for node in B.nodes()]

# Preparar la información para hoverinfo
node_text = []
node_sizes = []  # Tamaños personalizados para los nodos de las criptomonedas

for node in B.nodes():
    if node in cryptos:
        # Obtener información de la criptomoneda
        node_info = df[df['symbol'] == node].iloc[0]
        volume = node_info['volume_24h']
        node_sizes.append(max(10, volume / 1e7))  # Reducir tamaño dividiendo por 1e7, con tamaño mínimo de 10
        node_text.append(f"Criptomoneda: {node_info['name']}<br>"
                         f"Símbolo: {node}<br>"
                         f"Capitalización de mercado: ${node_info['market_cap']}<br>"
                         f"Volumen: {node_info['volume_24h']}")
    else:
        # Tamaño fijo para categorías
        node_sizes.append(30)  # Tamaño fijo para categorías
        # Para categorías, contar cuántas criptomonedas están conectadas a esa categoría
        crypto_count = len(df[df['category'] == node])
        node_text.append(f"Categoría: {node}<br>"
                         f"Criptomonedas asociadas: {crypto_count}")

# Crear la figura para los nodos con hoverinfo detallado
node_trace = go.Scatter(
    x=x_nodes, y=y_nodes,
    mode='markers+text',
    text=[node for node in B.nodes()],
    textposition='top center',
    hoverinfo='text',
    hovertext=node_text,  # Información mostrada al pasar el cursor
    marker=dict(
        showscale=True,
        colorscale='Blues',
        size=node_sizes,  # Tamaños basados en el volumen para criptomonedas
        color=['skyblue' if node in cryptos else 'lightgreen' for node in B.nodes()],
        line_width=2
    )
)

# Crear las aristas con ponderación (color y estilo uniformes)
edge_traces = []  # Lista para múltiples trazas de aristas

for edge in B.edges(data=True):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]

    # Crear una traza individual para cada arista
    edge_trace = go.Scatter(
        x=[x0, x1, None],
        y=[y0, y1, None],
        line=dict(width=1, color='gray'),  # Línea fina y color uniforme
        hoverinfo='none',
        mode='lines'
    )
    edge_traces.append(edge_trace)

# Configurar la figura final
fig = go.Figure(data=edge_traces + [node_trace],
                layout=go.Layout(
                    title='Grafo Criptomonedas-Categorías con Tamaños Basados en Volumen',
                    title_x=0.5,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=0, l=0, r=0, t=50),
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
               )

# Mostrar la figura
fig.show()

In [4]:
import pandas as pd
import networkx as nx
import plotly.graph_objects as go

# Cargar dataset
url = '/content/1000----proyectos_criptos_categorias.csv'
df = pd.read_csv(url)

# Definir rangos de precios para agrupar las criptomonedas
bins = [ float('-inf'), 0.01, 5, 20, 200, float('inf')]  # Cambia los valores según las necesidades del dataset
labelsPriceGroup = ['Muy Bajo','Bajo', 'Medio', 'Alto', 'Muy Alto']

# Crear una nueva columna 'price_group' para agrupar precios en categorías
df['price_group'] = pd.cut(df['price'], bins=bins, labels=labelsPriceGroup, include_lowest=True)

# Eliminar filas con valores NaN en la columna 'price_group' (si las hubiera)
df = df.dropna(subset=['price_group'])

# Crear un grafo bipartito
B = nx.Graph()

# Conjuntos de nodos: símbolos y grupos de precios
symbols = df['symbol'].unique()


# Añadir nodos al grafo
B.add_nodes_from(symbols, bipartite=0)  # Nodos símbolos
B.add_nodes_from(labelsPriceGroup, bipartite=1)  # Nodos grupos de precios

# Añadir aristas entre símbolos y grupos de precios
for index, row in df.iterrows():
    B.add_edge(row['symbol'], row['price_group'])  

# Posicionamiento de los nodos usando bipartite_layout para plotly
pos = nx.bipartite_layout(B, symbols)

# Extraer posiciones para plotly
x_nodes = [pos[node][0] for node in B.nodes()]
y_nodes = [pos[node][1] for node in B.nodes()]

# Crear las aristas (conexiones) del grafo
edge_x = []
edge_y = []

for edge in B.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])  # None para cortar la línea entre las conexiones
    edge_y.extend([y0, y1, None])

# Crear la figura para las aristas
edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=1, color='gray'),
    hoverinfo='none',
    mode='lines'
)

# Preparar información para hovertext
node_text = []
for node in B.nodes():
    if node in symbols:
        # Obtener la información de la criptomoneda
        crypto_info = df[df['symbol'] == node].iloc[0]
        node_text.append(f"Criptomoneda: {crypto_info['name']}<br>"
                         f"Símbolo: {node}<br>"
                         f"Precio: ${crypto_info['price']:.10f}<br>"
                         f"Volumen: {crypto_info['volume_24h']}<br>"
                         f"Categoría: {crypto_info['category']}<br>"
                         f"Grupo de Precio: {crypto_info['price_group']}")  # Añadir el grupo de precio
    else:
        # Para los grupos de precios, contar cuántas criptomonedas pertenecen y el rango de precios
        crypto_count = len(df[df['price_group'] == node])
        if node == 'Muy Bajo':
            price_range = "$0 - $0.009"
        elif node == 'Bajo':
            price_range = "$0.01 - $4"
        elif node == 'Medio':
            price_range = "$5 - $19"
        elif node == 'Alto':
            price_range = "$20 - $199"
        else:
            price_range = "$200 a más"
        node_text.append(f"Grupo de Precio: {node}<br>"
                         f"Criptomonedas: {crypto_count}<br>"
                         f"Rango de Precio: {price_range}")

# Crear la figura para los nodos
node_trace = go.Scatter(
    x=x_nodes, y=y_nodes,
    mode='markers+text',
    text=[node for node in B.nodes()],
    textposition='top center',
    hoverinfo='text',
    hovertext=node_text,  # Información que se mostrará al pasar el cursor
    marker=dict(
        showscale=True,
        colorscale='YlGnBu',
        size=20,
        color=[0 if node in symbols else 1 for node in B.nodes()],
        colorbar=dict(
            thickness=15,
            title='Tipo de Nodo',
            xanchor='left',
            titleside='right'
        ),
        line_width=2
    )
)

# Configurar la figura final
fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                    title='Grafo Bipartito Símbolos-Grupos de Precios con Datos Detallados',
                    title_x=0.5,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=0, l=0, r=0, t=50),
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
               )

# Mostrar la figura
fig.show()
